In [8]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Conv2DTranspose

# Definisci le dimensioni desiderate per le immagini
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 3  # Numero di canali dell'immagine RGB

# Percorso delle cartelle contenenti le immagini e le maschere
data_folder = "../training_data"
images_folder = os.path.join(data_folder, "horses")
masks_folder = os.path.join(data_folder, "masks")

# Ottieni l'elenco dei nomi dei file delle immagini
train_ids = os.listdir(images_folder)

# Inizializza gli array X e y
X = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

# Ciclo su tutte le immagini
for i, image_name in enumerate(train_ids):
    # Leggi l'immagine
    image_path = os.path.join(images_folder, image_name)
    image = cv2.imread(image_path)
    # Riscalala alle dimensioni desiderate
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    # Salva l'immagine nell'array X
    X[i] = image
    
    
    # Leggi la maschera corrispondente
    mask_path = os.path.join(masks_folder, image_name)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    # Riscalala alle dimensioni desiderate
    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
    # Aggiungi una dimensione per il canale (1 canale per la maschera)
    mask = np.expand_dims(mask, axis=-1)
    # Salva la maschera nell'array y
    y[i] = mask

# Adesso X contiene le immagini e y contiene le rispettive maschere, entrambe riscalate alle dimensioni desiderate

# splitto per training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)   

### ENCODER ###

In [9]:
inputs = tf.keras.layers.Input(shape=(None, None, 3))
def encoder(inputs):
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    p5 = MaxPooling2D(pool_size=(2, 2))(c5)

    c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p5)
    c6 = Dropout(0.3)(c6)
    c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
    return c6

### DECODER ###

In [10]:
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, Add
num_classes = 1
def decoder(c6):
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Add()([u7, c7])

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Add()([u8, c8])

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Add()([u9, c9])

    u10 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c9)
    c10 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u10)
    c10 = Add()([u10, c10])

    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(c10)
    return outputs


### MODEL ###

In [11]:
from tensorflow.keras.utils import plot_model

# Supponendo che 'inputs' sia il tensore di input del modello
encoder_outputs = encoder(inputs)
outputs = decoder(encoder_outputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Visualizzazione del modello
plot_model(model, show_shapes=True)



AttributeError: module 'pydot' has no attribute 'InvocationException'

In [ ]:
from tensorflow.keras.utils import plot_model

# Supponendo che 'inputs' sia il tensore di input del modello
encoder_outputs = encoder(inputs)
outputs = decoder(encoder_outputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
#tf.keras.utils.plot_model(model, "model.png",show_shapes=True)


In [ ]:
# Compila il modello prima di utilizzarlo per l'addestramento
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Definisci i callback
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=15, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]

# Addestramento del modello
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=100, callbacks=callbacks)


Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 522ms/step - accuracy: 0.5760 - loss: 15.3036 - val_accuracy: 0.7404 - val_loss: 0.6435
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 475ms/step - accuracy: 0.6901 - loss: 0.6090 - val_accuracy: 0.7410 - val_loss: 0.5848
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 523ms/step - accuracy: 0.7216 - loss: 0.5321 - val_accuracy: 0.7442 - val_loss: 0.5390
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 521ms/step - accuracy: 0.7383 - loss: 0.4783 - val_accuracy: 0.7454 - val_loss: 0.5196
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 459ms/step - accuracy: 0.7407 - loss: 0.4542 - val_accuracy: 0.7454 - val_loss: 0.4595
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 513ms/step - accuracy: 0.7512 - loss: 0.4252 - val_accuracy: 0.7454 - val_loss: 0.5109
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 517ms/step - accuracy: 0.7540 - loss: 0.4100 - val_accuracy: 0.7454 - val_loss: 0.4864
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 493ms/step - accuracy: 0.7476 - loss: 0.4112 - val_ac